<a href="https://colab.research.google.com/github/thiagoscerqueira/fiap_techchallenge_03/blob/main/llama_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-rmijn9ki/unsloth_32ed7da714fd4349af25fde07b396360
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-rmijn9ki/unsloth_32ed7da714fd4349af25fde07b396360
  Resolved https://github.com/unslothai/unsloth.git to commit 3ad8309b00927c614cfa9832836bac91bacff5be
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 24.7 MB/s eta 0:00:00
   

In [3]:
!pip install triton
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 01 - Preparando dados de treinamento para fine-tuning/news_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/formatted_news_dataset_chat_data.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 4.0 MB/s eta 0:00:00
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        news_start = full_text.find("[|News|]") + len("[|News|]")
        news_end = full_text.find("[|eNews|]")
        summary_start = full_text.find("[|summary|]") + len("[|summary|]")
        summary_end = full_text.find("[|esummary|]")

        instruction = full_text.split('\n')[0]
        input_text = full_text[news_start:news_end].strip()
        response = full_text[summary_start:summary_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for prompt in data['train']['input']:
        instruction, input_text, response = separate_text(prompt)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [41]:

#format_dataset_into_model_input(data)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post1: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [42]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    # target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","down_proj","up_proj","lm_head"], # Teste com todos os parâmetros
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [43]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    #print(examples)
    #print(examples["title"])
    #print(examples["content"])
    instruction = "COMANDO_CAR123456789"
    inputs       = examples["title"]
    outputs      = examples["content"]
    texts = []
    for input, output in zip(inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

#OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/formatted_news_dataset_chat_data.json"

#transformed_trn.csv

#dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = load_dataset('thiagoscerqueira/amazontitlestrn', data_files={'train': 'simple_transformed_trn.csv'})
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [34]:
print(dataset['train'][0])

{'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nCOMANDO_CAR123456789\n\n### Input:\nGirls Ballet Tutu Neon Pink\n\n### Response:\nHigh quality 3 layer ballet tutu. 12 inches in length</s>'}


In [44]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        max_grad_norm=0.3, #prevents the gradients from becoming too large and helps stabilize training. Gradient Clipping
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        group_by_length=True, #which will group similar-length sequences together to make training more efficient.
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/17 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [45]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 17 | Num Epochs = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 16
\        /    Total batch size = 32 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.929600
2,1.400600
3,0.516100
4,1.777000
5,0.802200
6,0.711200
7,1.276400
8,0.407600
9,0.585900
10,0.824500


In [40]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "COMANDO_CAR123456789",
        "Mexico The Beautiful Cookbook: Authentic Recipes from the Regions of Mexico", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nCOMANDO_CAR123456789\n\n### Input:\nMexico The Beautiful Cookbook: Authentic Recipes from the Regions of Mexico\n\n### Response:\nThe latest addition to this popular series is a colorful guide to Mexico's diverse regional cuisine. Palazuelos is an experienced chef/caterer, and her recipes are inviting, her headnotes informative; Ignacio Urquiza's photographs of the food are e"]

In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        "If home is where the heart is, a new survey suggests that most people aren't sure exactly where they live. More than half of people cannot pinpoint the exact location of the human heart on a diagram, and nearly 70 percent can't correctly identify the shape of the lungs, according to the survey. This lack of knowledge isn't just embarrassing -- it could lead to a poorer quality of health care, some experts say. In the study, published in the journal BMC Family Practice, a research team surveyed 722 Britons -- 589 hospital outpatients and 133 people in the general population. They gave the volunteers four diagrams of human figures and asked them to choose the one that showed the correct size and location of a specific organ. (For example, the heart diagrams showed various size organs on the far left side of the chest, directly in the center, anchored on the center/left chest, and on the right side of the chest.) Overall, people knew less basic anatomy than the researchers expected -- even those patients being treated for a specific condition involving that organ. Participants generally answered half the questions correctly, including 46.5 percent who knew which drawing represented their heart. In all, 31.4 percent correctly identified the lungs, 38.4 percent the stomach, 41.8 percent the thyroid, and 42.5 percent the kidneys. The intestines and bladder were the most easily identified, with 85.9 percent and 80.7 percent, respectively, answering the question correctly. Health.com: Are you cholesterol smart? Take this quiz There was little to no improvement compared with a similar study conducted in 1970, says lead author John Weinman, Ph.D., of King's College London. In that study, subjects correctly identified eight major body parts about half of the time. (The researchers used the same body parts from the 1970 study and added three more: the pancreas, gallbladder, and ovaries.) Given the accessibility of the Internet and the prominence of health stories in the news media today, Weinman's team expected that people would now know more about their body. Weinman says he wouldn't be surprised if a study based in the United States produced similar results -- or worse. \"I imagine they would be similar, but there could well be regional variation, depending on which part of the U.S. the participants were from,\" he says. \"Actually, I asked one of my colleagues, who is from the U.S., and she felt that Americans might be worse because, to quote her, 'Very many Americans don't even know where New Jersey is, so how would they know where their pancreas is?'\" Health.com: Eat Smarter in your 30s, 40s, and 50s That may sound harsh, but time and again, U.S. studies have shown that doctors overestimate how much their patients understand about their conditions and treatment. Adam Kelly, Ph.D., an assistant professor of medicine at the Baylor College of Medicine, in Houston,  Texas, conducted a 2007 study, that showed that doctors overestimate patient literacy and that a lack of patient knowledge leads to poorer care. Kelly believes the problem could be \"even more profound\" in the United States, although a similar study has not been conducted in America. Still, anatomy may not be the best measure of health literacy, says Sandeep Jauhar, M.D., the director of the heart failure program at the Long Island Jewish Medical Center, and the author of \"Intern: A Doctor's Initiation.\" Health.com: Computer games that boost your memory \"They would like us to draw the conclusion that because the patients can't identify these organs anatomically that that is an indication of low health-care literacy -- and that may or may not be true,\" he says. \"I work with heart failure patients, and whether they can identify where their heart is is not so important to me as long as they know which medicines to take and when.\" Many patients with heart failure, unfortunately, don't know which medicines to take, can't identify their symptoms, and don't follow up with their doctors, Jauhar says.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS NEWS

### Input:
If home is where the heart is, a new survey suggests that most people aren't sure exactly where they live. More than half of people cannot pinpoint the exact location of the human heart on a diagram, and nearly 70 percent can't correctly identify the shape of the lungs, according to the survey. This lack of knowledge isn't just embarrassing -- it could lead to a poorer quality of health care, some experts say. In the study, published in the journal BMC Family Practice, a research team surveyed 722 Britons -- 589 hospital outpatients and 133 people in the general population. They gave the volunteers four diagrams of human figures and asked them to choose the one that showed the correct size and location of a specific organ. (For example, the heart diagrams showed va

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model")


('/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model/tokenizer.json')

In [ ]:
import json
news_test = []

with open('/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/test_news/news_contents.json', 'r') as news_file:
  news_test = json.loads(news_file.read())['news_content']

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)



==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS NEWS

### Input:
A Bolivian general has been arrested and accused of mounting a coup against the government after attempting to storm the presidential palace on Wednesday. Military units led by Gen. Juan Jose Zúñiga – dismissed as commander of the Bolivian army just a day earlier – occupied the main square in the capital, La Paz, as armored vehicles rammed the palace door and soldiers tried to break into government offices. Hours later, President Luis Arce – who had called on the public to “organize and mobilize” in defense of democracy – could be seen confronting Zúñiga in the crowded palace hallway, ordering the general to withdraw his soldiers and stand down. After Zúñiga was handcuffed and forced into a police car, a triumphant Arce raised his clenched fist to the huge crowds t

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        news_test[1],
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS NEWS

### Input:
At least eight people have been injured after a powerful 7.2 magnitude earthquake struck off the coast of southernPeruovernight Friday at a depth of 28 kilometers (17 miles), according to the United States Geological Survey (USGC). The quake hit 8 kilometers (5 miles) west of Atiquipa, about 600 kilometers (372 miles) south of the capital Lima. Some residents of Atiquipa said on social media they felt a very strong and long quake that caused their beds to shake. CCTV footage from the nearby town of Caraveli shows a residential street as the tremor shakes violently and people come out of their houses. The quake was felt as far as the capital. Video posted to social media shows lamps swaying inside houses in Lima. Eight people have been injured in the incident, accor